In [1]:
%autosave 1

Autosaving every 1 seconds


In [2]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [3]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/3', creation_time=1721665888781, experiment_id='3', last_update_time=1721665888781, lifecycle_stage='active', name='mlops-tiny-experiment_v2', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1721421668701, experiment_id='2', last_update_time=1721421668701, lifecycle_stage='active', name='mlops-tiny-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1721336176115, experiment_id='1', last_update_time=1721336176115, lifecycle_stage='active', name='nyc-taxi-mlops', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1721333618813, experiment_id='0', last_update_time=1721333618813, lifecycle_stage='active', name='Default', tags={}>]

# Create a New Experiment

In [4]:
client.create_experiment(name="mlops-tiny-experiment_v3")

'4'

# Search for Active Experiments and show 5 results

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(experiment_ids="1",
                   filter_string="",
                   run_view_type=ViewType.ACTIVE_ONLY,
                   max_results=5,
                   order_by=["metrics.rmse ASC"])

In [6]:
for run in runs:
    print(f"run_id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run_id: 8dcd91d4430f40f690a42de78d0dc078, rmse: 6.2849
run_id: a07911fefe92406797edd87a11107cd2, rmse: 6.2883
run_id: c451ebb305614d0cbc388037ae536d99, rmse: 6.2921
run_id: e8bb7a2b8d3941d0a92be0796a29d7b4, rmse: 6.2943
run_id: 8e582df710544613b474c1e26654c92b, rmse: 6.3041


# Use a filter to search for runs that meet criteria

In [7]:

runs = client.search_runs(experiment_ids="1",
                   filter_string="metrics.rmse < 6.29",
                   run_view_type=ViewType.ACTIVE_ONLY,
                   max_results=5,
                   order_by=["metrics.rmse ASC"])

In [8]:
for run in runs:
    print(f"run_id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run_id: 8dcd91d4430f40f690a42de78d0dc078, rmse: 6.2849
run_id: a07911fefe92406797edd87a11107cd2, rmse: 6.2883


# Register a New Model

In [9]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [10]:
run_id = "834a8fef4167423e8f96e882ed180ca0"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '5' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1721668667041, current_stage='None', description=None, last_updated_timestamp=1721668667041, name='nyc-taxi-regressor', run_id='834a8fef4167423e8f96e882ed180ca0', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/834a8fef4167423e8f96e882ed180ca0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

# Transition Model Stages

In [15]:
model_name = "nyc-taxi-regressor"
model_version = 5
latest_versions = client.get_latest_versions(name=model_name)

/tmp/ipykernel_30390/1507317951.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [12]:
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 3, stage: Production
version: 5, stage: None


In [16]:
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage="Production",
    archive_existing_versions=False
)

/tmp/ipykernel_30390/4135783601.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1721668667041, current_stage='Production', description=None, last_updated_timestamp=1721668718035, name='nyc-taxi-regressor', run_id='834a8fef4167423e8f96e882ed180ca0', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/834a8fef4167423e8f96e882ed180ca0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

In [17]:
from datetime import datetime

new_stage = "Production"
date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"

)

<ModelVersion: aliases=[], creation_timestamp=1721668667041, current_stage='Production', description='The model version 5 was transitioned to Production on 2024-07-22', last_updated_timestamp=1721668730605, name='nyc-taxi-regressor', run_id='834a8fef4167423e8f96e882ed180ca0', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/834a8fef4167423e8f96e882ed180ca0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

# Put Everything Together

In [18]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer

from sklearn.metrics import root_mean_squared_error


dv = DictVectorizer()

In [33]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.fit_transform(train_dicts)

def test_model(name, stage, X_test, y_test):
    model =mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [67]:
# Error using March data. Use Feb instead for notebook
df = read_dataframe( './data/green_tripdata_2021-02.parquet')

In [68]:
run_id = "12c5e8f9432b4b76976820e8e3b3e026"

In [69]:
client.download_artifacts(run_id=run_id, path="preprocessor", dst_path=".")

'/workspaces/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [70]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [71]:
X_test = preprocess(df, dv)

In [72]:
target = "duration"
y_test = df[target].values

In [73]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


CPU times: user 756 ms, sys: 3.72 ms, total: 760 ms
Wall time: 573 ms


{'rmse': 25.06129607844584}